# Gecko dataset modelling

In [143]:
# Import required packages
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import r2_score, mean_squared_error

In [169]:
def prepare_data(path: Path, features: list, target: list=None):
    """Load data from the given path, select required features, preprocess the data
    and if needed select the target
    """
    dataset = pd.read_csv(path)
    if not target:
        # Test dataset lacks this column, but it is present in the training set
        dataset["ohe_parentspecies_decane_toluene"] = 0.0

    x = dataset[features]
    
    # One hot encoding of parentspecies column has Nan values, 
    # I decided to keep the rows with Nans and set each of the ohe* variables to one 
    x[x[["ohe_parentspecies_apin",
    "ohe_parentspecies_apin_decane",
    "ohe_parentspecies_apin_decane_toluene",
    "ohe_parentspecies_apin_toluene",
    "ohe_parentspecies_decane",
    "ohe_parentspecies_decane_toluene"]].isna()] = 1.0

    # id column is not a meanigful feature
    x = x.drop("id", axis=1)
    if target:
        y = dataset[target]
        return x, y
    else:
        return x
    
def transform_predictions(predictions):
    """Reverse transformation applied to the target variable. First unscales the variable based on the mean and std values from the training set
    then applies exp function. Verified to work properly on the train set."""
    df = pd.read_csv("data/train.csv")
    orig_log_mean = np.log(df["pSat_Pa"]).mean()
    orig_log_std = np.log(df["pSat_Pa"]).std()
    unscaled = unscale(predictions, orig_log_mean, orig_log_std)
    return np.exp(unscaled)

def unscale_orig_target(target):
    """Unscale the value of pSat_Pa to obtain the original values. Verified to work properly for the train set."""
    df = pd.read_csv("data/train.csv")
    orig_mean = df["pSat_Pa"].mean()
    orig_std = df["pSat_Pa"].std()
    return unscale(target, orig_mean, orig_std)

def unscale(variable, mean, std):
    return std * variable + mean

def scale(variable, mean, std):
    return (variable - mean) / std

def save_test_predictions(test_predictions, save_path, original_test_file):
    original_test = pd.read_csv(original_test_file)
    original_test["target"] = test_predictions
    original_test["Id"] = original_test["id"]
    result = original_test[["Id", "target"]]
    result.to_csv(save_path, index=False)

In [170]:
features = ['id', 'NumOfAtoms', 'NumOfC', 'NumOfO', 'NumOfN',
       'NumHBondDonors', 'NumOfConfUsed', 'cc', 'ccco',
       'hydroxyl_alkl', 'aldehyde', 'ketone', 'carboxylic_acid', 'ester',
       'ether_alicyclic', 'nitrate', 'nitro', 'aromatic_hydroxyl',
       'carbonylperoxynitrate', 'peroxide', 'hydroperoxide',
       'carbonylperoxyacid', 'nitroester', 'trans_NumOfConf',
       'trans_MW',
       'ohe_parentspecies_apin', 'ohe_parentspecies_apin_decane',
       'ohe_parentspecies_apin_decane_toluene',
       'ohe_parentspecies_apin_toluene', 'ohe_parentspecies_decane',
       'ohe_parentspecies_decane_toluene']
target = ['pSat_Pa', 'trans_pSat_Pa']

data_dir = Path("data")
train_data_path = data_dir / "gecko_ml.csv"
test_data_path = data_dir / "gecko_ml_test.csv"
original_x, original_y = prepare_data(train_data_path, features, target=target)
x, y = prepare_data(train_data_path, features, target=target)
x_test = prepare_data(test_data_path, features)

x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.3)

y_train_org = y_train[target[0]]
y_train_trans = y_train[target[1]]
y_train_log10 = np.log10(unscale_orig_target(y_train_org))
y_train_log10_scaled = scale(y_train_log10, y_train_log10.mean(), y_train_log10.std())

y_val_org = y_val[target[0]]
y_val_trans = y_val[target[1]]
y_val_log10 = np.log10(unscale_orig_target(y_val_org))
y_val_log10_scaled = scale(y_val_log10, y_val_log10.mean(), y_val_log10.std())

x, y = shuffle(x, y)
y_org = y[target[0]]
y_trans = y[target[1]]
y_log10 = np.log10(unscale_orig_target(y_org))
y_log10_scaled = scale(y_log10, y_log10.mean(), y_log10.std())


/tmp/ipykernel_32217/1962207925.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[x[["ohe_parentspecies_apin",
/tmp/ipykernel_32217/1962207925.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[x[["ohe_parentspecies_apin",
/tmp/ipykernel_32217/1962207925.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[x[["oh

    
### Dataset columns descriptions

##### Original variables that were scaled:
- **id** - A unique molecule index used in naming files

- **pSat_Pa** - The saturation vapour pressure of the molecule calculated by COSMOtherm (Pa)
- **MW** - The molecular weight of the molecule (g/mol).
- **NumOfAtoms** - The number of atoms in the molecule
- **NumOfC** - The number of carbon atoms in the molecule
- **NumOfO** - The number of oxygen atoms in the molecul
- **NumOfN** - The number of nitrogen atoms in the molecule
- **NumHBondDonors** - “The number of hydrogen bond donors in the molecule, i.e. hydrogens bound to oxygen.”
- **NumOfConf** - The number of stable conformers found and successfully calculated by COSMOconf.
- **NumOfConfUsed** - The number of conformers used to calculate the thermodynamic properties.

- **cc** - The number of non-aromatic C=C bounds found in the molecule.
- **ccco** - The number of “C=C-C=O” structures found in non-aromatic rings in the molecule.
- **hydroxyl_alkl** - The number of the alkylic hydroxyl groups found in the molecule.
- **aldehyde** - The number of aldehyde groups in the molecule.
- **ketone** - The number of ketone groups in the molecule.

- **carboxylic_acid** - The number of carboxylic acid groups in the molecule.
- **ester** - The number of ester groups in the molecule.
- **ether_alicyclic** - The number of alicyclic ester groups in the molecule.
- **nitrate** - The number of alicyclic nitrate groups in the molecule
- **nitro** - The number of nitro ester groups in the molecule
- **aromatic_hydroxyl** - The number of alicyclic aromatic hydroxyl groups in the molecule.
- **carbonylperoxynitrate** - The number of carbonylperoxynitrate groups in the molecule.
- **peroxide** - The number of peroxide groups in the molecule
- **hydroperoxide** - The number of hydroperoxide groups in the molecule.
- **carbonylperoxyacid** - The number of carbonylperoxyacid groups found in the molecule
- **nitroester** - The number of nitroester groups found in the molecule
- **parentspecies** - Either “decane”, “toluene”, “apin” for alpha-pinene or a combination of these connected by an underscore to indicate ambiguous descent. In 243 cases, the parent species is “None” because it was not possible to retrieve it.
    

##### Transformed or created variables (also scaled):
                
- **trans_NumOfConf**,    # NumOfConf to the power of 0.3           
- **trans_MW**,           # Square root of trans_MW              
- **trans_pSat_Pa**,      # log transformed pSat_Pa                  
- **new_MW_hydroxyl_alkl_interaction**, #interaction term between MW and hydroxyl_alkl
- **new_polarity_score**,  #polarity with assigning different weights to functional group features               
- **new_num_pca_1**,      #First principal component for all variable with "Num" in names               
- **new_num_pca_2**,      #Second principal component for all variable with "Num" in names                  

#### Variables for the one hot encoding for parentspecies:
- **ohe_parentspecies_apin**,               # parentspecies category apin
- **ohe_parentspecies_apin_decane**,        # parentspecies category decane
- **ohe_parentspecies_apin_decane_toluene**,# parentspecies category apin_decane_toluene
- **ohe_parentspecies_apin_toluene**,       # parentspecies category apin_toluene
- **ohe_parentspecies_decane**,             # parentspecies category decane
- **ohe_parentspecies_decane_toluene**,     # parentspecies category decane_toluene

# Model selection


In [130]:
lin_reg = LinearRegression()
rf = RandomForestRegressor()
svr = SVR()
gbr = GradientBoostingRegressor()

In [173]:
scores = cross_val_score(lin_reg, x, y_log10, cv=10, scoring='r2')
print(scores.mean())

0.7095661569053759


In [174]:
scores = cross_val_score(rf, x, y_log10, cv=10, scoring='r2')
print(scores.mean())

0.7165979435672526


In [176]:
scores = cross_val_score(svr, x, y_log10, cv=10, scoring='r2')
print(scores.mean())

0.7475439245520384


In [175]:
scores = cross_val_score(gbr, x, y_log10, cv=10, scoring='r2')
print(scores.mean())


0.7240028565283071


In [171]:
rf.fit(x, y_log10_scaled)

RandomForestRegressor()

In [177]:
svr.fit(x, y_log10)

SVR()

In [178]:

preds = svr.predict(x_test)
answ = unscale(preds, y_log10.mean(), y_log10.std())
save_test_predictions(preds, "svr_log10.csv", test_data_path)